### Import Libraries & Create Spark Session

In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp
from pyspark.sql.functions import count


# Create Spark session
spark = SparkSession.builder \
    .appName("Reddit Silver Layer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()


### Load Data from Bronze Layer

In [0]:
df_bronze = spark.read.format("delta").table("big_data_analytics_v.big_data_analytics_sesssion_v.bronze_reddit_posts")

### Clean the Data

In [0]:
# Drop rows with null values in critical columns (post_id, title, and score)
df_cleaned = df_bronze.dropna(subset=["post_id", "title", "score"])

# Log the number of rows after cleaning for validation
print(f"Number of rows after cleaning: {df_cleaned.count()}")

# Step 2: Ensure uniqueness by dropping duplicate rows based on the 'post_id' column
df_unique = df_cleaned.dropDuplicates(["post_id"])

# Log the number of unique rows for validation
print(f"Number of unique rows: {df_unique.count()}")

# Step 3: Perform additional transformations
df_transformed = df_unique.withColumn(
    "created_at", 
    unix_timestamp("created_utc").cast("timestamp")  # Make sure 'created_utc' is a valid timestamp
).filter(col("score") > 10)

# Step 4: Aggregate data by author to see the number of posts per author
df_author_count = df_transformed.groupBy("author").agg(count("post_id").alias("post_count"))

df_transformed = df_transformed.drop("created_utc")

Number of rows after cleaning: 2300
Number of unique rows: 109


### Insert Data into Silver Table in Unity Catalog

In [0]:
# Create or replace the temporary view with the transformed data (optional)
df_transformed.createOrReplaceTempView("silver_reddit_posts_temp")

# Insert data into the Silver Delta table (appending data)
df_transformed.write.format("delta").mode("append").option("streaming", "true").saveAsTable("big_data_analytics_v.big_data_analytics_sesssion_v.silver_reddit_posts")

# Log the completion of the insert process
print("Data inserted into Silver table successfully!")

Data inserted into Silver table successfully!
